In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')
import RamanGraphene as rg


In [2]:
x_axis_300_g = np.loadtxt('calibrated_axis_300gr.dat')

lambda_laser = 632.8
x_axis_300_g_raman = rg.convert_nm_to_raman(x_axis_300_g, lambda_laser)

reference_flake_1 = np.loadtxt('Raman_suspended_graphene_sample_6_633nm_1mW_300gr_mm_RT_300s_flake_1_1.dat')[:,1]
map_flake_1 = rg.get_map_data('../09_12_2024/Flake_1/LRVI_H12P1_1-4__2024-12-09_17-25-27_tot.dat')

reference_flake_2 = np.loadtxt('Raman_suspended_graphene_sample_6_633nm_1mW_300gr_mm_RT_300s_flake_2_1.dat')[:,1]
map_flake_2 = rg.get_map_data('../09_12_2024/Flake_2/LRVI_H12P1_1-4_map2_2024-12-09_19-51-12_tot.dat')

omega_G = 1581
omega_2D = 2630

In [8]:
#Example plot for the intro
%matplotlib qt
x_example, y_example = rg.select_range(x_axis_300_g_raman, reference_flake_1, 1400, 2900)
x_example, y_example = rg.remove_offset(x_example, y_example, 10)

pos_G, pos_D, pos_2D = 1581.6, 2460.0, 2633.3

plt.figure(figsize=(9, 6))
plt.plot(x_example, y_example / np.max(y_example), linewidth=2.5)
plt.axvline(x=pos_G, color='gray', linestyle='--')
plt.axvline(x=pos_D, color='gray', linestyle='--')
plt.axvline(x=pos_2D, color='gray', linestyle='--')

plt.text(pos_G, 1.21, 'G', ha='center', va='bottom', color='black', fontsize=20)
plt.text(pos_D, 1.21, 'D+D"', ha='center', va='bottom', color='black', fontsize=20)
plt.text(pos_2D, 1.21, '2D', ha='center', va='bottom', color='black', fontsize=20)

plt.xlabel('Raman shift (cm$^{-1}$)', fontsize=20)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
plt.ylabel('Intensity (a.u.)', fontsize=20)

plt.ylim(-0.04, 1.19)

plt.savefig('../../Figures/Graphene_Raman_Spectrum_example.pdf')
plt.show()


In [10]:
#Fit for the reference spectrum of sample 6, flake 1

x_min = 1400
x_max = 2800

x_ROI = x_axis_300_g_raman[(x_axis_300_g_raman > x_min) & (x_axis_300_g_raman < x_max)]
y_ROI = reference_flake_1[(x_axis_300_g_raman > x_min) & (x_axis_300_g_raman < x_max)]

results_G, results_2D, params1 = rg.raman_Gr_fit(x_ROI, y_ROI, omega_G, omega_2D)

In [13]:
rg.print_fit_results(params1, 3)

G peak: 
 off_G = -108.378 ± 11.800 
 ω_G = 1582.168 ± 0.053 cm⁻¹ 
 Γ_G = 14.749 ± 0.174 cm⁻¹ 
 I_G = 148145.545 ± 1435.341
2D peak: 
 off_2D = -30.308 ± 16.960 
 ω_2D- = 2632.598 ± 0.060 cm⁻¹, ω_2D+ = 2645.616 ± 0.060 cm⁻¹ 
 Γ_2D = 20.940 ± 0.104 cm⁻¹ 
 I_2D = 1135942.034 ± 2708.461 
 I_2D-/I_2D+ = 2.381 ± 0.005


In [12]:
x_plot_G = np.linspace(min(results_G[:,0]), max(results_G[:,0]), 1000)
y_plot_G = rg.lorentzian(x_plot_G, params1[0][0], params1[0][1], params1[0][2], params1[0][3])

In [25]:
#Plot for the fit of the reference spectrum of sample 6, flake 1, G peak
%matplotlib qt

plt.rcParams.update({'font.size': 18})
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

# Plot for the fit of the reference spectrum of sample 6, flake 1, G peak
ax1.scatter(results_G[:,0], results_G[:,1], label='Data', color='k')
ax1.plot(x_plot_G, y_plot_G, color='red', label='Lorentzian Fit')
ax1.set_xlabel('Raman shift (cm$^{-1}$)')
ax1.set_ylabel('Intensity (a.u.)')
ax1.legend(['Data', 'Lorentzian fit'])

# Plot of the residuals for the fit of the reference spectrum of sample 6, flake 1, G peak
ax2.plot(results_G[:,0], (results_G[:,2]-results_G[:,1]) / rg.find_x0_y0(results_G[:,0], results_G[:,1], omega_G)[1])
ax2.set_xlabel('Raman shift (cm$^{-1}$)')
ax2.set_ylabel('Residuals / (Max. Intensity)')
ax2.text(0.05, 0.95, f'$\omega_G$: {params1[0][1]:.2f} cm$^{-1}$\n'
                    f'$\Gamma_G$: {params1[0][2]:.2f} cm$^{-1}$\n',
         transform=ax2.transAxes, verticalalignment='top')


plt.tight_layout()
plt.savefig('../../Figures/Flake_1_G_peak.pdf')
plt.show()

<>:18: SyntaxWarning: invalid escape sequence '\o'
<>:19: SyntaxWarning: invalid escape sequence '\G'
<>:18: SyntaxWarning: invalid escape sequence '\o'
<>:19: SyntaxWarning: invalid escape sequence '\G'
/tmp/ipykernel_18409/3104485211.py:18: SyntaxWarning: invalid escape sequence '\o'
  ax2.text(0.05, 0.95, f'$\omega_G$: {params1[0][1]:.2f} cm$^{-1}$\n'
/tmp/ipykernel_18409/3104485211.py:19: SyntaxWarning: invalid escape sequence '\G'
  f'$\Gamma_G$: {params1[0][2]:.2f} cm$^{-1}$\n',


In [30]:
x_plot_2D = np.linspace(min(results_2D[:,0]), max(results_2D[:,0]), 1000)
y_plot_2D_0 = rg.db_basko(x_plot_2D, params1[1][0], params1[1][1], params1[1][2], params1[1][3], params1[1][4], params1[1][5])
y_plot_2D_1 = rg.basko(x_plot_2D, params1[1][0], params1[1][1], params1[1][2], params1[1][3] * params1[1][4])
y_plot_2D_2 = rg.basko(x_plot_2D, params1[1][0], params1[1][1] + params1[1][5], params1[1][2], params1[1][3]*(1 - params1[1][4]))


In [45]:
#Plot for the fit of the reference spectrum of sample 6, flake 1, 2D peak
plt.rcParams.update({'font.size': 18})

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9,10))

ax1.scatter(results_2D[:,0], results_2D[:,1], label = 'Data', color = 'k')
ax1.plot(x_plot_2D, y_plot_2D_0, color='red', label='Mod. Lorentzian Fit')
ax1.plot(x_plot_2D, y_plot_2D_1, color='green', label = r'$2D_+$ subfeature')
ax1.plot(x_plot_2D, y_plot_2D_2, color='blue', label = r'$2D_-$ subfeature')
ax1.set_xlabel('Raman shift (cm$^{-1}$)')
ax1.set_ylabel('Intensity (a.u.)')
ax1.legend()

ax2.plot(results_2D[:,0], (results_2D[:,2] - results_2D[:,1]) / rg.find_x0_y0(results_2D[:,0], results_2D[:,1], omega_2D)[1])
ax2.set_xlabel(r'Raman shift ($cm^{-1}$)')
ax2.set_ylabel('Residuals / (Max. Intensity)')
ax2.text(0.05, 0.95, r'$\omega_{2D}$:'f' {params1[1][1]:.2f} cm$^{-1}$\n'
                    r'$\Gamma_{2D}$:'f' {2 * np.sqrt(2 ** (2 / 3) - 1) * params1[1][2]:.2f} cm$^{-1}$\n',
         transform=ax2.transAxes, verticalalignment='top')
plt.tight_layout()
plt.savefig('../../Figures/Flake_1_2D_peak.pdf')
plt.show()

In [32]:
#Fitting the whole map
params_map_G = []
params_map_2D = []
for i in range(np.shape(map_flake_1)[1]):
    try:
        tmp1, tmp2, tmp_params = rg.raman_Gr_fit(x_axis_300_g_raman, map_flake_1[:,i], omega_G, omega_2D)

        params_map_G.append(tmp_params[0])
        params_map_2D.append(tmp_params[1])
    except:
        pass

params_map_G = np.array(params_map_G)
params_map_2D = np.array(params_map_2D)

/home/andres-rodriguez/TdG/Graphene/Stephane_samples_last/11_12_2024/../../RamanGraphene.py:98: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_DB_Basko, _ = curve_fit(db_basko, x2D, y2D, p0_2D)


In [33]:
%matplotlib qt

#G vs 2D peak position
plt.plot(params_map_G[:,1], params_map_2D[:,1], 'o')
plt.title('Sample 6, flake 1: 2D vs G peak position', fontsize=20)
plt.xlabel('G peak position (cm$^{-1}$)', fontsize=14)
plt.ylabel('2D peak position (cm$^{-1}$)', fontsize=14)
plt.show()
